In [1]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from requests_html import HTML

________________________________________

In [2]:
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z']
url = "https://soundcloud.com/discover"

In [3]:
browser = webdriver.Chrome()
browser.get(url)

In [4]:
# Lưu user
users = []
# Lưu track
tracks = []
# Lưu playlist
playlists = []

**Chuyển từ list sang csv/tsv**

In [5]:
def list_to_tsv_csv(listItem, outputFile):
    df = pd.DataFrame(listItem)
    df.to_csv(outputFile, sep='\t')

**Bấm để đồng ý cookies**

In [6]:
# Đồng ý cookies
def accept_cookies():
    soup = BeautifulSoup(browser.page_source)
    cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
    cookies.click()

**Kéo xuống đến hết trang**

In [7]:
# nên check lại bởi vì vài lần chưa scroll hết :<
def scroll_down():
    # Loop 2 lần cho chắc ăn (có thể cho 3 lần)
    for i in range(2):
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        lastCount = 0
        match = False
        
        while(match == False):
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage: 
                match = True
            else: 
                lastCount = lenOfPage

**Lấy tất cả users và tracks của một playlist**

In [8]:
def get_all_users_tracks():
    scroll_down()
    
    soup = BeautifulSoup(browser.page_source)
    
    search = soup.find_all("div", "soundTitle__titleContainer")
    
    for producer in search:
        user = producer.find("a", "soundTitle__username").get_text()
        track = producer.find("a", "soundTitle__title").get_text()
        if user not in users:
            users.append(user)
        if track not in tracks:
            tracks.append(track)

In [9]:
def get_all_playlists():
    scroll_down()
    
    soup = BeautifulSoup(browser.page_source)
    
    search = soup.find_all("div", "soundTitle__titleContainer")
    
    for producer in search:
        playlist = producer.find("a", "soundTitle__title").get_text()
        if playlist not in playlists:
            users.append(user)

In [10]:
accept_cookies()

In [ ]:
for letter in alphabet:
    url = "https://soundcloud.com/search/sounds?q=" + letter
    browser.execute_script('window.open("{link}");'.format(link = url));
    browser.switch_to.window(browser.window_handles[1])
    time.sleep(2)
    get_all_users_tracks()
    time.sleep(1)
    browser.close()
    browser.switch_to.window(browser.window_handles[0])

In [ ]:
for letter in alphabet:
    url = "https://soundcloud.com/search/sets?q=" + letter
    browser.execute_script('window.open("{link}");'.format(link = url));
    browser.switch_to.window(browser.window_handles[1])
    time.sleep(2)
    get_all_playlists()
    time.sleep(1)
    browser.close()
    browser.switch_to.window(browser.window_handles[0])

In [ ]:
browser.quit()

**Lưu data vào file tsv**

In [ ]:
outputFile = "Crawl_data/user.csv"
list_to_tsv_csv(users, outputFile)
outputFile = "Crawl_data/track.csv"
list_to_tsv_csv(tracks, outputFile)
outputFile = "Crawl_data/playlist.csv"
list_to_tsv_csv(playlists, outputFile)

In [ ]:
print(len(users))
print(len(tracks))
print(len(playlists))